In [1]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from statsmodels.graphics.gofplots import qqplot
import tensorflow as tf
from sklearn import preprocessing, svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
import sklearn.metrics as metrics
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from imblearn.over_sampling import SMOTE
from collections import Counter
from scipy.stats import randint, loguniform
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import PowerTransformer
from xgboost import XGBClassifier
from sklearn.svm import SVC
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scipy.stats import reciprocal, uniform
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from skopt import BayesSearchCV
from scipy.stats import randint as sp_randint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import missingno as mso
import seaborn as sns
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from statsmodels.graphics.gofplots import qqplot
import tensorflow as tf
from sklearn import preprocessing, svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
import sklearn.metrics as metrics
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from imblearn.over_sampling import SMOTE , ADASYN
from collections import Counter
from scipy.stats import randint, loguniform
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import PowerTransformer
from xgboost import XGBClassifier
from sklearn.svm import SVC
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scipy.stats import reciprocal, uniform
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from skopt import BayesSearchCV
from scipy.stats import randint as sp_randint
from sklearn.metrics import *
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd   
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

# Data Loading

In [2]:
app_df = pd.read_csv('/kaggle/input/credit-card-approval-prediction/application_record.csv')
credit_df = pd.read_csv('/kaggle/input/credit-card-approval-prediction/credit_record.csv')

# Data Merging

**We will merge both datasets to link every user with his/her credit history**

In [3]:
#sorting IDs
app_df = app_df.sort_values('ID')
credit_df = credit_df.sort_values('ID')
#Map the status to bad or good
credit_df['STATUS_BINARY'] = credit_df['STATUS']
credit_df = credit_df.replace({'STATUS_BINARY' :{'C' : 'Good','X' : 'Good','0' : 'Bad', '1' : 'Bad','2' : 'Bad','3' : 'Bad','4' : 'Bad','5' : 'Bad'}})
result_df = credit_df.value_counts(subset=['ID', 'STATUS_BINARY']).unstack(fill_value=0).reset_index()
#decide the user is good or bad based on his/her history majority
result_df.loc[(result_df['Good'] / result_df['Bad'] >= 1), 'Status'] = 1
result_df.loc[(result_df['Good'] / result_df['Bad'] < 1 ), 'Status'] = 0
result_df['Status'] = result_df['Status'].astype(int)
df = app_df.merge(result_df, how='inner', on=['ID'])
#define the rate of good debts for each user
df['rate'] = (df['Good']+0.1) / (df['Bad']+0.1)
df.drop(['Good','Bad'],axis=1,inplace = True);
#Renaming columns to more friendly names
df.columns = ['ID', 'Gender', 'Car', 'Realty', 'Children', 'Income', 'Income_Type',
                        'Education_Type', 'Family_Status', 'Housing_Type',  'Age', 'Years_Experience' , 'Mobile_Phone', 'Work_Phone', 
                        'Phone', 'Email', 'Job_Title', 'Total_Family' , 'Status','rate']
df.head(5)

# EDA

**How many good and bad history users ?**

In [4]:
df.groupby('Status')['Status'].count().plot(kind='pie', y='Status')

**what is the majority gender of users ?**

In [5]:
df.groupby('Gender')['Gender'].count().plot(kind='pie', y='Gender')

# Features effect on the decision and comparison with the rate

In [6]:
df.groupby('Income_Type')['rate'].mean().plot(kind = 'bar')

**it looks like that Students have the worst rates**

In [7]:
df.groupby('Family_Status')['rate'].mean().plot(kind = 'bar')

**Single has bad rate compared to married**

In [8]:
df.groupby('Car')['rate'].mean().plot(kind = 'bar')

**Users with cars have slightly better rates**

In [9]:
df.groupby('Realty')['rate'].mean().plot(kind = 'bar')

In [10]:
df.groupby('Total_Family')['rate'].mean().plot(kind = 'bar')

**Families with 6 & 7 members have good rate !!**

# Features distribution

In [11]:
df.hist(column='Age');

In [12]:
df.hist(column='Income');

In [13]:
df.hist(column='Years_Experience');

**Some features have skewed distribution so we will normalize the features in the next steps**

**Corelation between features**

In [14]:
corr = df.corr()
plt.figure(figsize = (10,10))
sns.heatmap(corr, cmap="Blues", annot=True)

**Based on Corr Heatmap**

**We will remove children (beacase high correlation with total_family)**

**We will remove Mobile_phone**

In [15]:
df.drop(['Children','Mobile_Phone'],axis=1,inplace=True)

In [16]:
df.info()

In [17]:
corr = df.drop('ID',axis=1).corr()
plt.figure(figsize = (10,10))
sns.heatmap(corr, cmap="Blues", annot=True)

**We can observe that Income and age plays a big factor**

# (Preprocessing)

# Handling missing values

In [18]:
df.isna().sum()

In [19]:
df = df.dropna()

# Handling Numerical values

**Convert  age and YOE to year format**

In [20]:
df['Age'] = df['Age']/(-1*365)
df['Years_Experience'] = df['Years_Experience']/(-1*365)
df.head(5)

# Handling Categorical Valus

**Label encoding**

In [21]:
Cat_features = ['Realty', 'Gender','Car','Income_Type','Education_Type','Family_Status','Housing_Type','Job_Title']

for col in Cat_features:
    print(df[col].unique())
    
df.info()

In [22]:
le = preprocessing.LabelEncoder()

for col in Cat_features:
    df[col] = le.fit_transform(df[col])

df.info()

**Final data**

In [23]:
df.describe().T

# Train Test split

In [24]:
from sklearn.model_selection import train_test_split
X = df.drop(['ID','Status'],axis = 1)
y = df['Status']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=123)

# Over-sampling

In [25]:
sm = ADASYN()
X_train, y_train = sm.fit_resample(X_train,y_train)

# Transformation

In [26]:
power = PowerTransformer(method='yeo-johnson')
X_train = power.fit_transform(X_train)
X_val = power.fit_transform(X_val)

# Scaling

In [27]:
normalizer = preprocessing.StandardScaler()
X_train = normalizer.fit_transform(X_train)
X_val = normalizer.fit_transform(X_val)

# Machine learning

**4 base models**

# RandomForestClassifier

In [28]:
model = RandomForestClassifier(random_state = 0)
model.fit(X_train, y_train)
y_predict = model.predict(X_val)
print("Accuracy = " , accuracy_score(y_val, y_predict))
print("F1 = " , f1_score(y_val, y_predict))

# XGBClassifier

In [29]:
model =  XGBClassifier()
model.fit(X_train, y_train)
y_predict = model.predict(X_val)
print("Accuracy = " , accuracy_score(y_val, y_predict))
print("F1 = " , f1_score(y_val, y_predict))

# SVMClassifier

In [30]:
model = SVC(C=1,gamma = 0.001)
model.fit(X_train, y_train)
y_predict = model.predict(X_val)
print("Accuracy = " , accuracy_score(y_val, y_predict))
print("F1 = " , f1_score(y_val, y_predict))

# MLPClassifier

In [31]:
model = MLPClassifier(verbose = 0 , max_iter = 46 , random_state = 0)
model.fit(X_train, y_train)
y_predict = model.predict(X_val)
print("Accuracy = " , accuracy_score(y_val, y_predict))
print("F1 = " , f1_score(y_val, y_predict))